In [21]:
from outils import *
from hog import hog
import numpy as np

In [93]:
D = 10
N = 100
h = hog(D,N)

In [94]:
h.jouer(h.strategie_aveugle(),h.strategie_aveugle())

(102, 83)

In [178]:
def moyenne(strategie1,strategie2,steps=1000):
    def f():
        score1,score2 = h.jouer(strategie1,strategie2)
        return 0 if score1 > score2 else 1
    
    a = np.zeros((steps))-1
    vfunc = np.vectorize(lambda x: f())
    
    return 1 - vfunc(a).sum()/steps

In [183]:
a = np.asarray([moyenne(h.strategie_aveugle(),h.strategie_aveugle()) for i in range(0,10)])
savetxt('data.csv', a, delimiter=',')


array([0.556, 0.528, 0.549, 0.541, 0.529, 0.523, 0.508, 0.53 , 0.511,
       0.555])

In [182]:
data = loadtxt('data.csv', delimiter=',')
print(data)